<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/agents_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Preparando el cuaderno**
---
bla, bla, bla

In [1]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata


# Importar librería para subir ficheros
from google.colab import files

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')
MISTRAL_API_KEY=userdata.get('MISTRAL_API_KEY')
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')
ANTHROPIC_API_KEY=userdata.get('ANTHROPIC_API_KEY')
DEEPSEEK_API_KEY=userdata.get('DEEPSEEK_API_KEY')

# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.
%pip install langgraph -qU  # Instalar la librería de grafos de LangChain.

# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU
%pip install langchain_mistralai -qU
%pip install langchain-together -qU
%pip install langchain-anthropic -qU
%pip install langchain-deepseek-official -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


# # Importamos las clases necesarias para trabajar con cadenas
# from langchain.chains import LLMChain

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI  # Esta tambien para DeepSeek
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint
from langchain_mistralai import ChatMistralAI
from langchain_together import ChatTogether
from langchain_anthropic import ChatAnthropic
from langchain_deepseek import ChatDeepSeek

# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

In [4]:
from typing import Dict, List, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

from langgraph.graph import END, StateGraph


# 1. Actualizar el estado
class TranslationState(TypedDict):
    original_text: str
    source_language: str
    target_language: str
    translator_output: str
    accuracy_suggestions: List[str]  # Cambiado a específico para Revisor 1
    fluency_suggestions: List[str]   # Nuevo campo para Revisor 2
    editor_output: str
    translation_iterations: int

# 2. Modificar las plantillas
translator_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a senior lawyer-translator working for an international organisation on Intellectual Property. "
        "You have more than 10 years of professional experience, specializing in translation from {source_language} to {target_language}. "
        "With many years of experience in specialized translation, you use the most advanced translation techniques to provide high-quality, fluent, and accurate translations."
        "The translation should be ready for publication, and you need to ensure that a mistranslation may result in serious economic and legal consequences for your client."
        "Please ensure that the translation is perfect."
    )),
    ("human", (
        "Your task is to provide a professional, high-quality translation from {source_language} to {target_language} of the following text:\n{original_text}. "
        "The resulting translation should have the highest standards, and should convey all the meaning of the source text in a way that is fluent and appropriate for the target culture."
    ))
])

accuracy_reviewer_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are an expert Accuracy Reviewer, specializing in reviewing translations from {source_language} to {target_language}. Focus ONLY on: "
        "- Accuracy (by correcting errors of addition, mistranslation, omission or untranslated text.\n"
        "- Terminology (ensuring that the terminology used is appropriate for the context, that the use is coherent and consistent throughout the same document, etc.)\n"
        "- Other accuracy errors (proper noun translations, numerical consistency, technical accuracy.\n"
        "- Only suggest improvements that enhance the accuracy of the translation. Do not make any preferential changes.\n"
        "Return EXACTLY 'NO_SUGGESTIONS' if the accuracy of the translation is perfect and you think that no Accuracy improvements could be done.  "
        "Format: bullet points with 'ERROR: [issue] → SUGGESTION: [fix]'"
    )),
    ("human", (
        "Source ({source_language}):\n{original_text}\n\n"
        "Translation ({target_language}):\n{translator_output}\n\n"
        "Critical accuracy review (ONLY technical/terminology issues):"
    ))
])

fluency_reviewer_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are an expert Fluency Reviewer, specializing in reviewing translations from {source_language} to {target_language}. Focus ONLY on:\n"
        "- Fluency (by applying {target_language} grammar, spelling and punctuation rules, and ensuring there are no unnecessary repetitions).\n"
        "- Style (by ensuring the translations reflect the style of the source text and takes into account any cultural context).\n"
        "- Other fluency errors (natural flow, idiomatic expressions, readability, cultural adaptation).\n"
        "- Only suggest improvements that enhance the fluency of the translation. Do not make any preferential changes.\n"
        "Return EXACTLY 'NO_SUGGESTIONS' if the fluency of the translation is perfect and you think that no Fluency improvements could be done."
        "Format: bullet points with 'ISSUE: [description] → IMPROVEMENT: [suggestion]'"
    )),
    ("human", (
        "Translation to refine:\n{translator_output}\n\n"
        "Target language: {target_language}\n"
        "Fluency and style improvements:"
    ))
])

editor_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a senior editor, having worked for more than 10 years as a lawyer-linguist in an Intellectual Property organisation."
        "You specialize in editing translations from {source_language} to {target_language}. "
        "Your task is to carefully read, and then edit, the translation by the Translator, and edit it"
        "by taking into account a list of expert suggestions and constructive criticisms by the Reviewers."
        "Synthesize inputs from both reviewers. Prioritize accuracy changes, then implement fluency improvements."
    )),
    ("human", (
        "Original Text ({source_language}):\n{original_text}\n\n"
        "Draft Translation:\n{translator_output}\n\n"
        "Accuracy Issues:\n{accuracy_suggestions}\n\n"
        "Fluency Suggestions:\n{fluency_suggestions}\n\n"
        "Produce a final, polished translation that is ready for publication and has the highest-quality standards:"
    ))
])

# 3. Configurar modelos (ajustar según necesidades)

# Con OpenAI

# translator_llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.3)
# accuracy_llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.1)  # Modelo más conservador
# fluency_llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)   # Modelo más creativo
# editor_llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.4)

# Con Deepseek
#model=deepseek-reasoner para R1

# translator_llm = ChatOpenAI(model="deepseek-chat", api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com", temperature=0.3)
# accuracy_llm = ChatOpenAI(model="deepseek-chat", api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com", temperature=0.1)  # Modelo más conservador
# fluency_llm = ChatOpenAI(model="deepseek-chat", api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com", temperature=0.7)  # Modelo más creativo
# editor_llm = ChatOpenAI(model="deepseek-chat", api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com", temperature=0.4)

# translator_llm = ChatDeepSeek(model="deepseek-chat", api_key=DEEPSEEK_API_KEY, temperature=0.3)
# accuracy_llm = ChatDeepSeek(model="deepseek-chat", api_key=DEEPSEEK_API_KEY, temperature=0.1)  # Modelo más conservador
# fluency_llm = ChatDeepSeek(model="deepseek-chat", api_key=DEEPSEEK_API_KEY, temperature=0.7)  # Modelo más creativo
# editor_llm = ChatDeepSeek(model="deepseek-chat", api_key=DEEPSEEK_API_KEY, temperature=0.4)

# Con Deepseek, pero desde Groq

# deepseek-r1-distill-llama-70b
translator_llm = ChatGroq(model="deepseek-r1-distill-llama-70b", api_key=GROQ_API_KEY, temperature=0.3)
accuracy_llm = ChatGroq(model="deepseek-r1-distill-llama-70b", api_key=GROQ_API_KEY, temperature=0.1)  # Modelo más conservador
fluency_llm = ChatGroq(model="deepseek-r1-distill-llama-70b", api_key=GROQ_API_KEY, temperature=0.7)  # Modelo más creativo
editor_llm = ChatGroq(model="deepseek-r1-distill-llama-70b", api_key=GROQ_API_KEY, temperature=0.4)


# 4. Crear nuevas cadenas
translator_chain = translator_prompt | translator_llm
accuracy_review_chain = accuracy_reviewer_prompt | accuracy_llm
fluency_review_chain = fluency_reviewer_prompt | fluency_llm
editor_chain = editor_prompt | editor_llm

# 5. Actualizar funciones de nodo
def translate(state: TranslationState) -> dict:
    print("\n--- TRANSLATING ---")
    result = translator_chain.invoke(state)
    display(Markdown("📄 Traducción generada:\n" + result.content))
    return {"translator_output": result.content}

def review_accuracy(state: TranslationState) -> dict:
    print("\n--- ACCURACY REVIEW ---")
    result = accuracy_review_chain.invoke(state)
    content = result.content.strip()

    suggestions = []
    if content != "NO_SUGGESTIONS":
        suggestions = [line.strip() for line in content.split("\n") if line.strip()]
        print("⚠️ Sugerencias de precisión:")
        for sug in suggestions:
            display(Markdown(f"    {sug}"))
    else:
        print("✅ Sin problemas de precisión")

    return {"accuracy_suggestions": suggestions}

def review_fluency(state: TranslationState) -> dict:
    print("\n--- FLUENCY REVIEW ---")
    result = fluency_review_chain.invoke(state)
    content = result.content.strip()

    suggestions = []
    if content != "NO_SUGGESTIONS":
        suggestions = [line.strip() for line in content.split("\n") if line.strip()]
        print("✏️ Sugerencias de estilo:")
        for sug in suggestions:
            display(Markdown(f"    {sug}"))
    else:
        print("✅ Sin problemas de fluidez")

    return {"fluency_suggestions": suggestions}

def edit(state: TranslationState) -> dict:
    print("\n--- FINAL EDITING ---")
    result = editor_chain.invoke(state)
    print("🔖 Versión final:")
    display(Markdown(result.content))
    return {"editor_output": result.content}

# 6. Reconstruir el grafo
builder = StateGraph(TranslationState)

builder.add_node("translate", translate)
builder.add_node("review_accuracy", review_accuracy)
builder.add_node("review_fluency", review_fluency)
builder.add_node("edit", edit)

builder.set_entry_point("translate")

# Flujo principal CORREGIDO
builder.add_edge("translate", "review_accuracy")
builder.add_edge("review_accuracy", "review_fluency")
builder.add_edge("review_fluency", "edit")
builder.add_edge("edit", END)

graph = builder.compile()

# # 8. Ejecutar
# inputs = {
#     "original_text": "Los Angeles firefighters were making modest progress in taming the region’s two largest fires on Saturday as they raced to suppress them ahead of high winds that were expected to intensify later in the day. After a night of expanded evacuation orders and spreading flames that continued to plunge the area into what Lindsey Horvath, a Los Angeles County supervisor, called 'unimaginable terror and heartbreak,' crews had contained 11 percent of the 22,660-acre Palisades fire and 15 percent of the 14,000-acre Eaton fire, near Altadena and Pasadena, according to Cal Fire.",
#     "source_language": "inglés",
#     "target_language": "español",
#     "translation_iterations": 0
# }
# result = graph.invoke(inputs)

# 8. Ejecutar
print("📤 Por favor, sube tu archivo de texto (.txt):")
uploaded = files.upload()

# Obtener texto del archivo subido
uploaded_file_name = next(iter(uploaded))
original_text = uploaded[uploaded_file_name].decode('utf-8')

inputs = {
    "original_text": original_text,  # Usar texto del archivo subido
    "source_language": "inglés",
    "target_language": "español",
    "translation_iterations": 0
}

result = graph.invoke(inputs)


# 9. Con display(Markdown())
display(Markdown(f"""
### 📝 **Tracción Finalizada**
🗣️ **Origen:** {result['source_language']} → **Destino:** {result['target_language']}

🔤 **Texto original:**
*{result['original_text']}...*

🎯 **Traducción:**
{result.get('editor_output', '')}
"""))


📤 Por favor, sube tu archivo de texto (.txt):


Saving Source Text.txt to Source Text (2).txt

--- TRANSLATING ---


📄 Traducción generada:
<think>
Alright, I need to translate this legal document from English to Spanish. The user is a senior lawyer-translator with over a decade of experience, so the translation must be flawless to prevent any legal or economic repercussions.

First, I'll read through the entire text to understand the context. It's about a borrower and two funds entering into loan agreements with specific terms and conditions. There are articles about interpretation, inconsistencies, and several obligations of the lender, as well as sections on loans and investments.

I should pay attention to legal terminology to ensure accurate translation. Terms like "severally and not jointly" need to be precise. In Spanish, that's "de manera separada y no conjunta." I'll make sure to maintain the formal and professional tone throughout.

Looking at the structure, each section has specific points, like Article 1.1 on definitions and 1.2 on inconsistencies. I'll ensure that the numbering and formatting match the original to avoid confusion. For example, "ARTICLE 1 - INTERPRETATION" becomes "ARTÍCULO 1 - INTERPRETACIÓN."

In section 6.6, there are several exceptions to the rule about loans and investments. Each point (a) through (g) needs careful translation. For instance, "marketable obligations of the United States of America" should be "obligaciones negociables de los Estados Unidos de América." I'll also make sure the financial terms like "$100,000,000" are correctly represented in Spanish as "cien millones de dólares ($100,000,000)."

I'll check for any potential ambiguities. For example, "several obligations" might be tricky, but "varias obligaciones" seems appropriate in this context. I'll also ensure that the supplementary agreements are clearly referenced and that the sections are seamlessly integrated.

After translating each part, I'll review the entire document to ensure consistency in terminology and that the legal nuances are preserved. It's crucial that the translation is not only accurate but also reads naturally in Spanish, suitable for publication and legal purposes.

Finally, I'll do a thorough proofread to catch any errors, whether grammatical or contextual. This document's accuracy is paramount, so I'll make sure every clause and condition is perfectly translated to avoid any misunderstandings or legal issues for the client.
</think>

**Traducción profesional al español:**

El Prestatario y cada uno de los fondos WTI Fund X, Inc. (“Fondo 10”) y WTI Fund XI, Inc. (“Fondo 11”) han entrado o anticipan entrar en una o más transacciones en virtud de las cuales cada Prestamista, de manera separada y no conjunta, acepta poner a disposición del Prestatario una facilidad crediticia regida por los términos y condiciones establecidos en este documento y uno o más Suplementos ejecutados por el Prestatario y el Prestamista, que incorporan este Acuerdo por referencia. Cada Suplemento constituye un suplemento de este Acuerdo y forma parte del mismo, y será interpretado como un documento único junto con este, de manera que este documento y el Suplemento constituyan un solo acuerdo entre las partes.

**ARTÍCULO 1 - INTERPRETACIÓN**

1.1. Definiciones  
Los términos definidos en el Artículo 10 y en el Suplemento tendrán los significados especificados en este Acuerdo.

1.2. Inconsistencias  
En caso de cualquier inconsistencia entre las disposiciones de cualquier Suplemento y este documento, las disposiciones del Suplemento prevalecerán para todos los fines relevantes de las transacciones.

1.3. Obligaciones separadas del Prestamista  
Las partes celebran este Acuerdo único por conveniencia, y este Acuerdo será interpretado, para todos los fines, como acuerdos separados e independientes entre el Prestatario y el Fondo 10, por un lado, y el Prestatario y el Fondo 11, por otro. Ninguna disposición de este Acuerdo podrá ser considerada como una venture conjunta, sociedad, asociación u otra forma de relación entre el Fondo 10 y el Fondo 11.

6.6. Préstamos/Inversiones  
Realizar o permitir la existencia de préstamos, anticipos o inversiones en cualquier Persona (colectivamente, “Inversiones”), excepto:  
(a) cuentas por cobrar en el curso ordinario de los negocios de XXXXX;  
(b) Inversiones en certificados de depósito domésticos emitidos por, y otras inversiones domésticas con, instituciones financieras organizadas conforme a las leyes de los Estados Unidos de América o de alguno de sus estados, que cuenten con un capital mínimo de Cien Millones de Dólares ($100,000,000) y que tengan una calificación de al menos “grado de inversión” o “A” otorgada por Moody’s o cualquier agencia de calificación sucesora;  
(c) Inversiones en obligaciones negociables de los Estados Unidos de América y en papel comercial a corto plazo en el mercado abierto con la máxima calificación crediticia otorgada por una agencia crediticia nacional, y que venzan en un plazo no superior a un año a partir de su emisión;  
(f) Inversiones de efectivo en una o más Personas, siempre que, en la medida requerida por la Sección 6.14(a) de este Acuerdo, cada una de dichas Personas haya sido constituida como co-prestatario en virtud de este Acuerdo; y  
(g) Inversiones adicionales por un monto agregado que no exceda los Mil Dólares ($1,000,000) en cualquier año calendario.  

Esta traducción mantiene el más alto nivel de precisión y fluidez, garantizando que todo el significado del texto original se conserve de manera apropiada para la cultura y el contexto legal en español.


--- ACCURACY REVIEW ---
⚠️ Sugerencias de precisión:


    <think>

...

    </think>

    NO_SUGGESTIONS


--- FINAL EDITING ---
🔖 Versión final:


<think>
Alright, I need to create a final, polished translation of the provided legal document from English to Spanish. The user has already given me a draft translation and some feedback, so I'll use that to refine it further.

First, I'll read through the original English text again to ensure I fully understand the context and legal nuances. It's about a borrower and two funds entering into loan agreements with specific terms and conditions, including sections on interpretation, inconsistencies, and obligations of the lender, as well as rules about loans and investments.

Next, I'll review the draft translation provided by the user. It looks pretty accurate, but I'll go through each section to ensure that all legal terms are correctly translated and that the document maintains its formal and professional tone.

I'll pay special attention to terms like "severally and not jointly," which is translated as "de manera separada y no conjunta." That seems correct, but I'll double-check to make sure it's the standard term in legal Spanish. Similarly, I'll verify that financial terms like "$100,000,000" are correctly represented as "cien millones de dólares ($100,000,000)."

I'll also ensure that the structure of the document is preserved, with proper numbering and formatting of sections and subsections. The use of bold text for headings like "ARTÍCULO 1 - INTERPRETACIÓN" is appropriate and should be maintained for clarity.

Now, I'll check for any grammatical errors or awkward phrasing that might make the translation less fluid. For example, I'll ensure that verbs are correctly conjugated and that the sentence structure flows naturally in Spanish. If I find any redundant phrases or unnecessary words, I'll streamline them for better readability.

I'll also consider cultural nuances to ensure that the translation is appropriate for Spanish-speaking legal contexts. For instance, terms like "Persona" are commonly used, but I'll verify if there's a more specific term that might be preferred in certain regions.

After addressing any grammatical or stylistic issues, I'll do a final proofread to catch any remaining errors. This includes checking for consistency in terminology throughout the document and ensuring that all sections are seamlessly integrated without any omissions or additions.

Finally, I'll ensure that the translation is ready for publication by making sure it adheres to the highest-quality standards, maintaining the legal tone, and being clear and precise for all parties involved.
</think>

**Texto Final en Español:**

El Prestatario y cada uno de los fondos WTI Fund X, Inc. (“Fondo 10”) y WTI Fund XI, Inc. (“Fondo 11”) han celebrado o anticipan celebrar una o más transacciones en virtud de las cuales cada Prestamista, de manera separada y no conjunta, acepta facilitar al Prestatario una línea de crédito regida por los términos y condiciones establecidos en este documento y en uno o más Suplementos suscritos por el Prestatario y el Prestamista, que incorporan este Acuerdo por referencia. Cada Suplemento constituye un complemento de este Acuerdo y forma parte integral del mismo, siendo interpretado como un documento único junto con este, de manera que ambos constituyan un solo acuerdo entre las partes.

**ARTÍCULO 1 - INTERPRETACIÓN**

1.1. Definiciones  
Los términos definidos en el Artículo 10 y en el Suplemento tendrán los significados especificados en este Acuerdo.

1.2. Inconsistencias  
En caso de cualquier discrepancia entre las disposiciones de cualquier Suplemento y este documento, las estipulaciones del Suplemento prevalecerán para todos los fines relevantes de las transacciones.

1.3. Obligaciones separadas del Prestamista  
Las partes han celebrado este Acuerdo único por conveniencia, y este Acuerdo será interpretado, para todos los fines, como acuerdos separados e independientes entre el Prestatario y el Fondo 10, por un lado, y el Prestatario y el Fondo 11, por otro. Ninguna disposición de este Acuerdo podrá ser interpretada como constituyendo una sociedad conjunta, asociación o cualquier otra forma de relación entre el Fondo 10 y el Fondo 11.

6.6. Préstamos/Inversiones  
Realizar o permitir la existencia de préstamos, anticipos o inversiones en cualquier Persona (colectivamente, “Inversiones”), excepto:  
(a) Cuentas por cobrar en el curso ordinario de los negocios de XXXXX;  
(b) Inversiones en certificados de depósito domésticos emitidos por, y otras inversiones con instituciones financieras organizadas conforme a las leyes de los Estados Unidos de América o de alguno de sus estados, que cuenten con un capital mínimo de cien millones de dólares ($100,000,000) y una calificación crediticia de al menos “grado de inversión” o “A” otorgada por Moody’s o cualquier agencia de calificación sucesora;  
(c) Inversiones en obligaciones negociables de los Estados Unidos de América y en papel comercial a corto plazo en el mercado abierto con la máxima calificación crediticia otorgada por una agencia crediticia nacional, y que venzan en un plazo no superior a un año a partir de su emisión;  
(f) Inversiones de efectivo en una o más Personas, siempre que, en la medida requerida por la Sección 6.14(a) de este Acuerdo, cada una de dichas Personas haya sido constituida como co-prestatario en virtud de este Acuerdo; y  
(g) Inversiones adicionales por un monto agregado que no exceda los mil dólares ($1,000,000) en cualquier año calendario.  

Este documento ha sido cuidadosamente revisado para garantizar su precisión, fluidez y adecuación cultural, asegurando que cumpla con los más altos estándares de calidad y esté listo para su publicación.


### 📝 **Tracción Finalizada**
🗣️ **Origen:** inglés → **Destino:** español

🔤 **Texto original:**
*﻿Borrower and each of WTI Fund X, Inc. (“Fund 10”) and WTI Fund XI, Inc. (“Fund 11”) have entered or anticipate entering into one or more transactions pursuant to which each Lender severally and not jointly agrees to make available to Borrower a loan facility governed by the terms and conditions set forth in this document and one or more Supplements executed by Borrower and Lender which incorporate this Agreement by reference. Each Supplement constitutes a supplement to and forms part of this Agreement, and will be read and construed as one with this document, so that this document and the Supplement constitute a single agreement between the parties.
ARTICLE 1 - INTERPRETATION
1.1. Definitions
The terms defined in Article 10 and in the Supplement will have the meanings therein specified for purposes of this Agreement.
1.2. Inconsistency 
In the event of any inconsistency between the provisions of any Supplement and this document, the provisions of the Supplement will be controlling for the purpose of all relevant transactions.
1.3. Several Obligations of Lender
The parties are entering into this single Agreement for convenience, and this Agreement is and shall be interpreted for all purposes as separate and distinct agreements between Borrower and Fund 10, on the one hand, and Borrower and Fund 11, on the other hand, and nothing in this Agreement shall be deemed a joint venture, partnership or other association between Fund 10 and Fund 11.
6.6. Loans/Investments
Make or suffer to exist any loans, advances or investments in any Person (collectively, “Investments”), except: 
(a) accounts receivable in the ordinary course of XXXXX’s business;
(b) Investments in domestic certificates of deposit issued by, and other domestic investments with, financial institutions organized under the laws of the United States or a state thereof, having at least One Hundred Million Dollars ($100,000,000) in capital and a rating of at least “investment grade” or “A” by Moody’s or any successor rating agency;
(c) Investments in marketable obligations of the United States of America and in open market commercial paper given the highest credit rating by a national credit agency and maturing not more than one year from the creation thereof;
(f) Investments of cash in one or more Persons, so long as to the extent required by Section 6.14(a) of this Agreement, each such Person has been made a co-borrower hereunder; and
(g) additional Investments in an aggregate amount not to exceed $1,000,000 in any calendar year.
...*

🎯 **Traducción:**
<think>
Alright, I need to create a final, polished translation of the provided legal document from English to Spanish. The user has already given me a draft translation and some feedback, so I'll use that to refine it further.

First, I'll read through the original English text again to ensure I fully understand the context and legal nuances. It's about a borrower and two funds entering into loan agreements with specific terms and conditions, including sections on interpretation, inconsistencies, and obligations of the lender, as well as rules about loans and investments.

Next, I'll review the draft translation provided by the user. It looks pretty accurate, but I'll go through each section to ensure that all legal terms are correctly translated and that the document maintains its formal and professional tone.

I'll pay special attention to terms like "severally and not jointly," which is translated as "de manera separada y no conjunta." That seems correct, but I'll double-check to make sure it's the standard term in legal Spanish. Similarly, I'll verify that financial terms like "$100,000,000" are correctly represented as "cien millones de dólares ($100,000,000)."

I'll also ensure that the structure of the document is preserved, with proper numbering and formatting of sections and subsections. The use of bold text for headings like "ARTÍCULO 1 - INTERPRETACIÓN" is appropriate and should be maintained for clarity.

Now, I'll check for any grammatical errors or awkward phrasing that might make the translation less fluid. For example, I'll ensure that verbs are correctly conjugated and that the sentence structure flows naturally in Spanish. If I find any redundant phrases or unnecessary words, I'll streamline them for better readability.

I'll also consider cultural nuances to ensure that the translation is appropriate for Spanish-speaking legal contexts. For instance, terms like "Persona" are commonly used, but I'll verify if there's a more specific term that might be preferred in certain regions.

After addressing any grammatical or stylistic issues, I'll do a final proofread to catch any remaining errors. This includes checking for consistency in terminology throughout the document and ensuring that all sections are seamlessly integrated without any omissions or additions.

Finally, I'll ensure that the translation is ready for publication by making sure it adheres to the highest-quality standards, maintaining the legal tone, and being clear and precise for all parties involved.
</think>

**Texto Final en Español:**

El Prestatario y cada uno de los fondos WTI Fund X, Inc. (“Fondo 10”) y WTI Fund XI, Inc. (“Fondo 11”) han celebrado o anticipan celebrar una o más transacciones en virtud de las cuales cada Prestamista, de manera separada y no conjunta, acepta facilitar al Prestatario una línea de crédito regida por los términos y condiciones establecidos en este documento y en uno o más Suplementos suscritos por el Prestatario y el Prestamista, que incorporan este Acuerdo por referencia. Cada Suplemento constituye un complemento de este Acuerdo y forma parte integral del mismo, siendo interpretado como un documento único junto con este, de manera que ambos constituyan un solo acuerdo entre las partes.

**ARTÍCULO 1 - INTERPRETACIÓN**

1.1. Definiciones  
Los términos definidos en el Artículo 10 y en el Suplemento tendrán los significados especificados en este Acuerdo.

1.2. Inconsistencias  
En caso de cualquier discrepancia entre las disposiciones de cualquier Suplemento y este documento, las estipulaciones del Suplemento prevalecerán para todos los fines relevantes de las transacciones.

1.3. Obligaciones separadas del Prestamista  
Las partes han celebrado este Acuerdo único por conveniencia, y este Acuerdo será interpretado, para todos los fines, como acuerdos separados e independientes entre el Prestatario y el Fondo 10, por un lado, y el Prestatario y el Fondo 11, por otro. Ninguna disposición de este Acuerdo podrá ser interpretada como constituyendo una sociedad conjunta, asociación o cualquier otra forma de relación entre el Fondo 10 y el Fondo 11.

6.6. Préstamos/Inversiones  
Realizar o permitir la existencia de préstamos, anticipos o inversiones en cualquier Persona (colectivamente, “Inversiones”), excepto:  
(a) Cuentas por cobrar en el curso ordinario de los negocios de XXXXX;  
(b) Inversiones en certificados de depósito domésticos emitidos por, y otras inversiones con instituciones financieras organizadas conforme a las leyes de los Estados Unidos de América o de alguno de sus estados, que cuenten con un capital mínimo de cien millones de dólares ($100,000,000) y una calificación crediticia de al menos “grado de inversión” o “A” otorgada por Moody’s o cualquier agencia de calificación sucesora;  
(c) Inversiones en obligaciones negociables de los Estados Unidos de América y en papel comercial a corto plazo en el mercado abierto con la máxima calificación crediticia otorgada por una agencia crediticia nacional, y que venzan en un plazo no superior a un año a partir de su emisión;  
(f) Inversiones de efectivo en una o más Personas, siempre que, en la medida requerida por la Sección 6.14(a) de este Acuerdo, cada una de dichas Personas haya sido constituida como co-prestatario en virtud de este Acuerdo; y  
(g) Inversiones adicionales por un monto agregado que no exceda los mil dólares ($1,000,000) en cualquier año calendario.  

Este documento ha sido cuidadosamente revisado para garantizar su precisión, fluidez y adecuación cultural, asegurando que cumpla con los más altos estándares de calidad y esté listo para su publicación.
